In [75]:
from pyspark.sql import Row
from pyspark.sql.types import *


# Functions

# Load test data

In [76]:
# Load data
wwine_rdd = sc.textFile("wine-data/winequality-white.csv")
wwine_rdd = wwine_rdd.map(lambda line: line.replace('"',''))
wwine_rdd = wwine_rdd.map(lambda line: line.replace(';',','))

In [77]:
# Get header
header = wwine_rdd.take(1)[0]
header = bb.split(',')
Wine = Row(*header)


In [79]:
# Create colection of Rows
data = wwine_rdd.map(lambda w: Wine(*w.split(',')))

# Create DataFrame
df = sqlContext.createDataFrame(data)

# Univariate selection